In [18]:
pip install cocoex

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install xgboost

  Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached xgboost-2.0.3-py3-none-manylinux2014_x86_64.whl (297.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [50]:
pip install pyearth

  Using cached pyearth-0.1.23-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached GDAL-3.8.3.tar.gz (802 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [85 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info
      writing /tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info/requires.txt
      writing top-level names to /tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-naaw9it0/GDAL.egg-info/SOURCES.txt'
      Traceback (most recent call last):
        File "/tmp/pip-install-pmq6n8t8/gdal_570d8a58329d4646a6ba5867d0567c99/setup.py", line 89, in fetch_config
          p = subprocess.Popen([co

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import ast
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
#from pyearth import Earth

ela_data = pd.read_csv('New_Transformed_File.csv')
ert_data = pd.read_csv('rel_ERT.csv')

merged_data = pd.merge(ela_data, ert_data, on=['fid', 'dim'])
target_columns = ["BSqi", "BSrr", "CMA-CSA", "fmincon", "fminunc", "HCMA",
                  "HMLSL", "IPOP400D", "MCS", "MLSL", "OQNLP", "SMAC-BBOB"]

def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return None

merged_data['features'] = merged_data['features'].apply(safe_eval)

merged_data = merged_data.dropna(subset=['features'])
max_len = merged_data['features'].apply(len).max()  # Find the maximum length of lists
#merged_data['features'] = merged_data['features'].apply(lambda x: x + [0] * (max_len - len(x)) if x is not None else [0] * max_len)

rf_model = RandomForestRegressor(n_estimators=500, random_state=42,max_features=int(max_len/3))
svm_model = SVR(kernel='rbf')
model_xgboost = XGBRegressor(objective='rank:pairwise', random_state=42)
dt_model = DecisionTreeRegressor()
#mars_model = Earth()

loo = LeaveOneOut()
mse_dict = {}
result_data = pd.DataFrame(columns=['mbse for RandomForest', 'mbse for xgboost','mbse for SVM','mbse for RPart','Algorithm'])
for target_column in target_columns:
    mse_dict[target_column] = {}
    # Extract features (X) and target variable (y)
    X = np.array(merged_data['features'].tolist())
    y = np.array(merged_data[target_column].tolist())
    
    mse_list = []
    mse_xgboost_list = []
    mse_svm_list = []
    mse_dt_list = []
    mse_mars_list = []
    for train_index, test_index in loo.split(merged_data):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
                   
                
        rf_model.fit(X_train, y_train)
        y_pred = rf_model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_list.append(mse)
                    
        model_xgboost.fit(X_train, y_train)
        y_pred_xgboost = model_xgboost.predict(X_train)
        mbse_xgboost = mean_squared_error(y_train, y_pred_xgboost)
        mse_xgboost_list.append(mbse_xgboost)
                    
        svm_model.fit(X_train, y_train)
        y_pred_svm = model_xgboost.predict(X_train)
        mbse_svm = mean_squared_error(y_train, y_pred_svm)
        mse_svm_list.append(mbse_svm)
                    
        dt_model.fit(X_train, y_train)
        y_pred_dt = dt_model.predict(X_train)
        mbse_dt = mean_squared_error(y_train, y_pred_dt)
        mse_dt_list.append(mbse_dt)
        
        #mars_model.fit(X_train, y_train)
        #y_pred_mars = dt_model.predict(X_train)
        #mbse_mars = mean_squared_error(y_train, y_pred_mars)
        #mse_mars_list.append(mbse_mars)
                    
    avg_mse = np.mean(mse_list)
    avg_mse_xgboost = np.mean(mse_xgboost_list)
    avg_mse_svm = np.mean(mse_svm_list)
    avg_mse_dt = np.mean(mse_dt_list)
    #avg_mse_mars = np.mean(mse_mars_list)

    result_data = pd.concat([result_data, pd.DataFrame({'mbse for RandomForest': [avg_mse],'mbse for xgboost': [avg_mse_xgboost],'mbse for SVM': [avg_mse_svm],'mbse for RPart': [avg_mse_dt],'Algorithm': [target_column]})], ignore_index=True)


result_data.to_csv("mbse_data_New.csv", encoding='utf-8', index=False)
